<a href="https://colab.research.google.com/github/SarahSaeedKhan/Tasks/blob/main/nueral_and_Fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#NAME: SARAH SAEED KHAN
#ROLLNO#FA21-RCE-006

#importing all the libraries
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets,transforms,models

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(1234)
if device == 'cuda':
    torch.cuda.manual_seed_all(1234)

In [ ]:

# directly downloading data from Kaggle  on to colab

import os
os.environ['KAGGLE_USERNAME']="sarahsaeedkhan"
os.environ['KAGGLE_KEY']="8a98c028cb9a51465d99b43c6a9a1eef"
!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 30.8MB/s]
 99% 537M/543M [00:03<00:00, 159MB/s]
100% 543M/543M [00:03<00:00, 142MB/s]
 94% 255M/271M [00:01<00:00, 155MB/s]
100% 271M/271M [00:01<00:00, 148MB/s]


In [ ]:
# Upload data and extract the contents from the zip file.
from zipfile import ZipFile

file_name = "/content/train.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
# printing the folder present in the directory and creating base directory
data_dir_list = os.listdir('/content/train')
#print(data_dir_list)

path, dirs, files = next(os.walk("/content/train"))
file_count = len(files)
print(file_count)

original_dataset_dir = '/content/train'
base_dir = '/content/PetImages'
os.mkdir(base_dir) #make base directory

25000


In [ ]:
#Create directory paths
# creating three folders from one(TRAIN TEST AND VALIDATION)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

In [ ]:
import shutil

fnames = ['cat.{}.jpg'.format(i) for i in range(8000)]
for fname in fnames:
   src = os.path.join(original_dataset_dir, fname)
   dst = os.path.join(train_cats_dir, fname)
   #print(src,dst)
   shutil.copyfile(src, dst)


fnames = ['cat.{}.jpg'.format(i) for i in range(8000,11000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(validation_cats_dir, fname)
  shutil.copyfile(src, dst)


fnames = ['cat.{}.jpg'.format(i) for i in range(11000, 12300)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(test_cats_dir, fname)
  shutil.copyfile(src, dst)


fnames = ['dog.{}.jpg'.format(i) for i in range(8000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(train_dogs_dir, fname)
  shutil.copyfile(src, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(8000,11000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(validation_dogs_dir, fname)
  shutil.copyfile(src, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(11000, 12300)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(test_dogs_dir, fname)
  shutil.copyfile(src, dst)

In [ ]:
#printing the number of images in each folder
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))

print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

total training cat images: 8550
total training dog images: 8550
total validation cat images: 3049
total validation dog images: 3049
total test cat images: 1300
total test dog images: 1300


In [ ]:
#data Augumentation
# resize the images to overcome overfitting our model
train_transforms =  transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])


test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
     transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
    ])

In [ ]:
##loading datasets
from PIL import Image
class dataset(torch.utils.data.Dataset):

    def __init__(self,file_list,transform=None):
        self.file_list = file_list
        self.transform = transform


    #dataset length
    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    #load an one of images
    def __getitem__(self,idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = img_path.split('/')[-1].split('.')[0]
        if label == 'dog':
            label=1
        elif label == 'cat':
            label=0

        return img_transformed,label

In [ ]:
train_data = datasets.ImageFolder(train_dir,transform=train_transforms)
val_data = datasets.ImageFolder(validation_dir,transform=test_transforms)
test_data = datasets.ImageFolder(test_dir,transform=test_transforms)

In [ ]:
print(len(train_data))#length of train data

17100


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = 1500,shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = 10,shuffle = True)
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size = 600,shuffle = True)

In [ ]:
print(len(train_data), len(train_loader))

17100 12


In [ ]:
print(len(val_data), len(val_loader))

6098 11


In [ ]:
#check our images shape
train_data[0][0].shape

torch.Size([3, 224, 224])

In [ ]:

# build Model
# 3 Convolution layer and 2 fully connected layer
# batchNormalization for limit overfitting
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn,self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3, padding=0,stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
            )

        self.layer3 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )


        self.fc1 = nn.Linear(3*3*64,10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10,2)
        self.relu = nn.ReLU()


    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = Cnn().to(device)#model summary
model.train()

Cnn(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=576, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
  (re

In [ ]:
optimizer = optim.Adam(params = model.parameters(),lr=0.001)#defining otimizers and hyper parameters
criterion = nn.CrossEntropyLoss()

In [ ]:
#trainning  the model
epochs = 10

for epoch in range(epochs):#train dataset
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in train_loader:
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = ((output.argmax(dim=1) == label).float().mean())
        epoch_accuracy += acc/len(train_loader)
        epoch_loss += loss/len(train_loader)

    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))


    with torch.no_grad():#validation dataset
        epoch_val_accuracy=0
        epoch_val_loss =0
        for data, label in val_loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output,label)


            acc = ((val_output.argmax(dim=1) == label).float().mean())
            epoch_val_accuracy += acc/ len(val_loader)
            epoch_val_loss += val_loss/ len(val_loader)

        print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch+1, epoch_val_accuracy,epoch_val_loss))

Epoch : 1, train accuracy : 0.5998055338859558, train loss : 0.6593274474143982
Epoch : 1, val_accuracy : 0.6482127904891968, val_loss : 0.6309503316879272
Epoch : 2, train accuracy : 0.6507499814033508, train loss : 0.627074122428894
Epoch : 2, val_accuracy : 0.6838188171386719, val_loss : 0.5993415713310242
Epoch : 3, train accuracy : 0.6823055148124695, train loss : 0.5985215306282043
Epoch : 3, val_accuracy : 0.6915460824966431, val_loss : 0.5818554759025574
Epoch : 4, train accuracy : 0.6968610882759094, train loss : 0.5765940546989441
Epoch : 4, val_accuracy : 0.7124922871589661, val_loss : 0.5654087662696838
Epoch : 5, train accuracy : 0.7189167141914368, train loss : 0.5568379163742065
Epoch : 5, val_accuracy : 0.7228881120681763, val_loss : 0.5418285131454468
Epoch : 6, train accuracy : 0.7168888449668884, train loss : 0.5495006442070007
Epoch : 6, val_accuracy : 0.7179839611053467, val_loss : 0.55143141746521
Epoch : 7, train accuracy : 0.7290555238723755, train loss : 0.5390

In [ ]:
epochs = 10#test dataset

for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in test_loader:
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = ((output.argmax(dim=1) == label).float().mean())
        epoch_accuracy += acc/len(test_loader)
        epoch_loss += loss/len(test_loader)

    print('Epoch : {}, test accuracy : {}, test loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))

Epoch : 1, test accuracy : 0.5934615135192871, test loss : 0.6669334769248962
Epoch : 2, test accuracy : 0.6496152877807617, test loss : 0.6256524324417114
Epoch : 3, test accuracy : 0.6365382671356201, test loss : 0.6279505491256714
Epoch : 4, test accuracy : 0.654999852180481, test loss : 0.6122523546218872
Epoch : 5, test accuracy : 0.6761535406112671, test loss : 0.5977608561515808
Epoch : 6, test accuracy : 0.6676921248435974, test loss : 0.5938078165054321
Epoch : 7, test accuracy : 0.6903840899467468, test loss : 0.5872085690498352
Epoch : 8, test accuracy : 0.7015383243560791, test loss : 0.5752211213111877
Epoch : 9, test accuracy : 0.7015381455421448, test loss : 0.5705613493919373
Epoch : 10, test accuracy : 0.7042301297187805, test loss : 0.5631405711174011
